In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -r /content/drive/MyDrive/TG/tg2_COVIDNet/requirements.txt

In [3]:
# train_ds_path = "../target_dataset/train"
# test_ds_path = "../target_dataset/test"
train_ds_path = "/content/drive/MyDrive/TG/target_dataset/train"
test_ds_path = "/content/drive/MyDrive/TG/target_dataset/test"

In [4]:
batch_size = 32
image_size = (240, 240)

from tensorflow.keras.preprocessing import image_dataset_from_directory

test_ds = image_dataset_from_directory(
    test_ds_path,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=False,
)

#Junta todos os batches
test_ds = test_ds.unbatch()


# TF tensors -> numpy array
test_ds = test_ds.as_numpy_iterator()

In [5]:
encoding = {"COVID-19": 2, "Normal": 0, "Pneumonia": 1}

In [ ]:
import cv2
import os
import numpy as np

X_train = []
y_train = []

for c in encoding:
    files = list(f for f in os.listdir(os.path.join(train_ds_path, c)) if os.path.isfile(os.path.join(train_ds_path, c, f)))
    for f in files:
        img = cv2.imread(os.path.join(train_ds_path, c, f), cv2.IMREAD_GRAYSCALE)
        # img = np.expand_dims(img, axis=2)
        img = cv2.resize(img, image_size)
        X_train.append(img)
        y_train.append(encoding[c])


In [ ]:
from random import shuffle

data_train = list(zip(X_train, y_train))
shuffle(data_train)
X_train, y_train = zip(*data_train)


In [ ]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)


In [ ]:

for train_index, test_index in skf.split(X_train, y_train):
    print("TRAIN:", train_index, "TEST:", test_index)

In [ ]:
from tensorflow.keras.models import load_model
import autokeras as ak

# model_path = "/Users/igorbeduin/Google Drive (beduinigor@gmail.com)/TG/tg2_COVIDNet/models/final_model.h5"
model_path = "/content/drive/MyDrive/TG/tg2_COVIDNet/models/final_model.h5"
model = load_model(model_path, custom_objects=ak.CUSTOM_OBJECTS)


In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical

x = tf.convert_to_tensor(X_train, np.float32)
y = to_categorical(y_train, 3)